In [2]:
import torch
from torch.utils.data import Dataset
from transformers import AutoProcessor
from PIL import Image
import os
import json

class Qwen2VLDataset(Dataset):
    def __init__(self, data_file, processor, image_size=(360, 420)):
        with open(data_file, 'r') as f:
            self.data = json.load(f) 
        self.processor = processor
        self.image_size = image_size  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        messages = item['messages']

        # Extract user message (image + prompt)
        user_message = messages[0]
        content = user_message['content']

        # Safely extract image or video info
        image_or_video = content[0]

        # Safe handling for text prompt
        if isinstance(content[1], dict) and 'text' in content[1]:
            text_prompt = content[1]['text']
        elif isinstance(content[1], list) and len(content[1]) > 0 and isinstance(content[1][0], dict) and 'text' in content[1][0]:
            text_prompt = content[1][0]['text']
        else:
            raise ValueError(f"Invalid format at index {idx}: content[1] = {content[1]}")

        # Load and process image or video
        if image_or_video['type'] == 'image':
            image_path = image_or_video['image']
            image = Image.open(image_path).convert("RGB")
            image = image.resize(self.image_size)
            image_input = self.processor(images=image, return_tensors="pt").pixel_values
        elif image_or_video['type'] == 'video':
            video_path = image_or_video['image']
            video_input = self.processor(video=video_path, return_tensors="pt").pixel_values
            image_input = video_input
        else:
            raise ValueError(f"Unsupported media type at index {idx}: {image_or_video['type']}")

        # Assistant message is the target output text
        assistant_message = messages[1]
        assistant_content = assistant_message['content']
        text_description = assistant_content

    # Tokenize the input
        inputs = self.processor(
            text=[text_prompt],
            images=image_input,
            padding=True,
            return_tensors="pt"
        )

        # Tokenize the output/label
        labels = self.processor.batch_encode_plus(
            [text_description],
            return_tensors="pt",
            padding=True
        ).input_ids

        # Move to GPU if available
        inputs = {k: v.squeeze().to("cuda") for k, v in inputs.items()}
        labels = labels.squeeze().to("cuda")

        return {
            'input_ids': inputs['input_ids'],
            'attention_mask': inputs['attention_mask'],
            'labels': labels
        }


# Example Usage:
# Initialize the processor and dataset
model_name = "Qwen/Qwen2-VL-2B-Instruct"
processor = AutoProcessor.from_pretrained(model_name)
train_dataset = Qwen2VLDataset(data_file="train_dataset.json", processor=processor)

# Check the first sample in the dataset
sample = train_dataset
print(*sample)

TypeError: argument of type 'NoneType' is not iterable

In [ ]:
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers.git


In [ ]:
messages

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print("".join(output_text))

In [ ]:
print("".join(output_text))

In [ ]:
torch.cuda.empty_cache()